In [1]:
%matplotlib widget

In [2]:
from pathlib import Path
import numpy as np
import flammkuchen as fl
import pandas as pd
import tifffile as tiff

from fimpylab import TwoPExperiment

import matplotlib.pyplot as plt
import json

from scipy.interpolate import interp1d
from scipy import signal

In [300]:
master = Path(r"\\Funes\Shared\experiments\E0075\e0075_v05")
files = list(master.glob("*_f*"))
fish = files[2]
path = fish / 'suite2p' / '0007'
path

WindowsPath('//Funes/Shared/experiments/E0075/e0075_v05/220629_f1/suite2p/0007')

In [301]:
# loading suite2p data:
suite2p_data = fl.load(path / 'data_from_suite2p_cells.h5')
coords = suite2p_data["coords"]
anat = suite2p_data["anatomy_stack"]
traces = suite2p_data["traces"]

OSError: ``\\Funes\Shared\experiments\E0075\e0075_v05\220629_f1\suite2p\0007\data_from_suite2p_cells.h5`` does not exist

In [302]:
# normalizing traces:
traces = traces.T
traces = ((traces - traces.mean(0)) / traces.std(0))
traces = traces.T

In [289]:
num_rows = 16
s_size = 1 / num_rows
choices = []
for x_pos in range(num_rows):
    curr_choice = [x_pos / num_rows, 0, s_size, 1]
    choices.append(curr_choice)
    curr_choice = [0, x_pos / num_rows, 1, s_size]
    choices.append(curr_choice)

In [290]:
# Getting stimulus information
metadata_file = list(path.glob("*_metadata.json"))[0]

with open(str(metadata_file), "r") as f:
     metadata = json.load(f)
        
stim = metadata["stimulus"]["log"]
len(stim)


640

In [291]:
# getting tail information:
beh_file = list(path.glob("*_behavior*"))[0]
beh_log = fl.load(beh_file)['data']

In [292]:
fs = int(metadata['imaging']['microscope_config']['scanning']['framerate'])
print("fs: ", fs)

pause_duration = stim[0]['duration'] * fs
stim_duration = stim[1]['duration'] * fs
print("pause: ", pause_duration / fs, " sec")
print("stim: ", stim_duration / fs, " sec")

fs:  2
pause:  2.0  sec
stim:  4.0  sec


In [293]:
n_options = 16 * 2
n_rep = [metadata['stimulus']['protocol']['receptive_fields']['v05_rows_cols_flashes_8x8']['n_trials']][0]
n_trials = (n_options * n_rep) 
position_list = np.zeros((n_trials, 4))
for_regs = np.zeros((n_options, n_trials * 2 + 1))

len_rec = np.shape(traces)[1]
print("len recording: ", len_rec)
regs = np.zeros((n_options, len_rec))
t1 = pause_duration

for i in range(1, n_trials * 2, 2):
    curr_trial = stim[i]['clip_mask']
    position_list[(i//2) - 1, :] = curr_trial
    
    for j in range(n_options):
        if curr_trial == choices[j]:
            for_regs[j, i-1] = 1
            regs[j, t1:(t1 + stim_duration)] = 1
    
    t1 = t1 + stim_duration + pause_duration 
print(i)

len recording:  3841
639


In [294]:
def exp_decay_kernel(tau, dt, len_rec):
    upsample = 10
    t = np.arange(len_rec * upsample) * dt / upsample
    
    decay = np.exp(-t / tau)
    decay /= np.sum(decay)
    return decay

In [295]:
# making one big figure:
from lotr.default_vals import REGRESSOR_TAU_S, TURN_BIAS

dt_imaging = 1 / fs
int_fact = 200
t_imaging = np.arange(traces.shape[1])/fs
num_traces, len_rec = np.shape(traces)

tau_fs = REGRESSOR_TAU_S * fs
kernel = np.exp(-np.arange(1000) / tau_fs)
t_imaging_int = np.arange(traces.shape[1]*int_fact)*dt_imaging/int_fact

In [296]:
regs_conv = np.zeros((n_options, len_rec))
regs_vals = np.zeros((n_options, num_traces))

for i in range(n_options):
    regs_conv[i] = np.convolve(regs[i], kernel)[:np.shape(traces)[1]]
    
    tmp_reg_vals = np.dot(traces, regs_conv[i]) - num_traces * np.mean(traces, 1) * np.mean(regs_conv[i])
    tmp_reg_vals /= (traces.shape[1] - 1) * np.std(traces, 1) * np.std(regs_conv[i])
    regs_vals[i] = tmp_reg_vals
    

In [297]:
fig_regs, ax_regs = plt.subplots(2, 1, figsize=(8, 6))

for i in range(n_options):
    ax_regs[0].plot(regs_conv[i] + (i * 20))

ax_regs[0].plot(beh_log.t * fs, beh_log.tail_sum*10 - 80, c='k')

ax_regs[0].set_xlim(0, len_rec)    
ax_regs[0].axis('off')
ax_regs[1].axis('off')
ax_regs[1].imshow(traces, extent=[0, 1500, 0, 500], cmap='coolwarm', vmin=-2, vmax=3)

file_name = 'RF_regressors_and_traces.jpg'
fig_regs.savefig(path / file_name, dpi=300)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [298]:
num_row = 6
fig_rf_reg, ax_rf_reg = plt.subplots(num_row, num_row, figsize=(10, 10), sharex=True, sharey=True)

for i in range(num_row*num_row):
    r = i // num_row
    c = np.mod(i, num_row)
    
    try:
        ax_rf_reg[r,c].scatter(coords[:, 1], coords[:, 2], c=regs_vals[i], cmap='coolwarm', s=2, vmin=-1, vmax=1)
        ax_rf_reg[r,c].imshow(np.rot90(anat[0],3), cmap='gray_r')
        ax_rf_reg[r,c].set_title('RF' + str(i))
    except:
        print("no plane")
    ax_rf_reg[r,c].axis('off')
    
    
file_name = 'RF_regressors_rois.jpg'
fig_rf_reg.savefig(path / file_name, dpi=300)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

no plane
no plane
no plane
no plane


In [299]:
fig_regs2, ax_regs2 = plt.subplots(1, 1, figsize=(8, 8))

for i in range(n_options):
    ax_regs2.plot(regs_conv[i] + (i * 20), c='royalblue')
    
    max_corr = np.nanmax(regs_vals[i])
    max_ind = np.argmax(regs_vals[i])
    
    ax_regs2.plot(traces[max_ind] + (i * 20), c='skyblue')

ax_regs2.set_xlim(0, len_rec)    
ax_regs2.axis('off')

file_name = 'RF_regressors_and_maxcorr_traces.jpg'
fig_regs2.savefig(path / file_name, dpi=300)

    
    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
mot_reg = fl.load(path / 'motor_regressors.h5')

In [ ]:
mot_reg

In [ ]:
############ Pooling across planes
fish = files[0]
num_row = 4
fig_rf_reg, ax_rf_reg = plt.subplots(num_row, 4, figsize=(7, 8), sharex=True, sharey=True)
path = fish / 'suite2p' / '0003'
suite2p_data = fl.load(path / 'data_from_suite2p_cells.h5')
anat = np.rot90(suite2p_data["anatomy_stack"][0], 1)
    
for i in range(16):
    r = i // num_row
    c = np.mod(i, num_row)

    ax_rf_reg[r,c].imshow(anat, cmap='gray_r')

                                          
dataset_path = fish / 'original'
with open(dataset_path / "stack_metadata.json") as f:
        stack_metadata = json.load(f)
num_planes = stack_metadata["shape_full"][1]

for plane in range(num_planes):
    
    try:
        plane_name = '000' + str(plane)
        path = fish / 'suite2p' / plane_name
        print(path)

        # loading suite2p data:
        suite2p_data = fl.load(path / 'data_from_suite2p_cells.h5')
    except:
        plane_name = '00' + str(plane)
        path = fish / 'suite2p' / plane_name
        print(path)

        # loading suite2p data:
        suite2p_data = fl.load(path / 'data_from_suite2p_cells.h5')
    coords = suite2p_data["coords"]
    traces = suite2p_data["traces"]

    # normalizing traces:
    traces = traces.T
    traces = ((traces - traces.mean(0)) / traces.std(0))
    traces = traces.T
    
    # Getting stimulus information
    metadata_file = list(path.glob("*_metadata.json"))[0]

    with open(str(metadata_file), "r") as f:
         metadata = json.load(f)

    stim = metadata["stimulus"]["log"]
    
    fs = int(metadata['imaging']['microscope_config']['scanning']['framerate'])

    pause_duration = stim[0]['duration'] * fs
    stim_duration = stim[1]['duration'] * fs
    
    n_options = 16
    n_rep = [metadata['stimulus']['protocol']['receptive_fields']['v02_square_flashes']['n_trials']][0]
    n_trials = (n_options * n_rep) 
    position_list = np.zeros((n_trials, 4))
    for_regs = np.zeros((16, n_trials * 2 + 1))

    len_rec = np.shape(traces)[1]
    regs = np.zeros((16, len_rec))
    t1 = pause_duration

    for i in range(1, n_trials * 2, 2):
        curr_trial = stim[i]['clip_mask']
        position_list[(i//2) - 1, :] = curr_trial

        for j in range(16):
            if curr_trial == choices[j]:
                for_regs[j, i-1] = 1
                regs[j, t1:(t1 + stim_duration)] = 1

        t1 = t1 + stim_duration + pause_duration 
    
    dt_imaging = 1 / fs
    int_fact = 200
    t_imaging = np.arange(traces.shape[1])/fs
    num_traces, len_rec = np.shape(traces)

    tau_fs = REGRESSOR_TAU_S * fs
    kernel = np.exp(-np.arange(1000) / tau_fs)
    t_imaging_int = np.arange(traces.shape[1]*int_fact)*dt_imaging/int_fact
    
    regs_conv = np.zeros((16, len_rec))
    regs_vals = np.zeros((16, num_traces))

    for i in range(16):
        regs_conv[i] = np.convolve(regs[i], kernel)[:np.shape(traces)[1]]

        tmp_reg_vals = np.dot(traces, regs_conv[i]) - num_traces * np.mean(traces, 1) * np.mean(regs_conv[i])
        tmp_reg_vals /= (traces.shape[1] - 1) * np.std(traces, 1) * np.std(regs_conv[i])
        regs_vals[i] = tmp_reg_vals
    

    for i in range(16):
        r = i // num_row
        c = np.mod(i, num_row)
        
        selected_coords = np.where(abs(regs_vals[i]) > 0.1)[0]
        try:
            ax_rf_reg[r,c].scatter(coords[selected_coords, 1], coords[selected_coords, 2], c=regs_vals[i, selected_coords], cmap='coolwarm', s=2, vmin=-1, vmax=1)#, alpha=0.7)
            ax_rf_reg[r,c].set_title('RF' + str(i))
        except:
            print("no plane")
        ax_rf_reg[r,c].axis('off')
        
    
    
file_name = 'RF_regressors_rois_all_planes_01.jpg'
fig_rf_reg.savefig(fish / file_name, dpi=300)
